In [70]:
import json
import pandas as pd

DATA_PATH = '../data/1832_v4_preprocessed.csv'
HIERARCHY_PATH = '../data/circle_packing_data.json'

In [71]:
rec = pd.read_csv(DATA_PATH)
rec.head()

,nom_rue,no_maison,chef_annee_naissance,chef_origine,chef_annee_arrivee,chef_vocation,page,division,chef_vocation_class,chef_origine_class
0,marterey,1,1764.0,tolochenaz,1786.0,charon,1,1-2,NaN,morges
1,marterey,2,NaN,rolle,1821.0,marechal-ferrant,1,1-2,artisanat,rolle
2,marterey,3,1785.0,pailly,NaN,lingere,1,1-2,service,moudon
3,marterey,3,NaN,gingins,NaN,rentier,1,1-2,rente,NaN
4,marterey,3,NaN,gingins,NaN,rentier,1,1-2,rente,NaN


In [72]:
rec = rec \
    .groupby(['division', 'nom_rue']) \
    .agg({'page': 'count'}) \
    .rename(columns={
        'page': 'value'
        })
rec.index.names = ['division', 'name']
rec.head()


value
division name            
1-2      beausite       1
         bugnon         4
         etraz         88
         georgette      4
         marterey     260

In [73]:
data = {"name": "divisions", "children": []}
for division, df in rec.groupby(level=0):
    data["children"].append({
        "name": division,
        "children": df.reset_index()[['name', 'value']].to_dict(orient="records")
    })

In [74]:
# Save data
with open(HIERARCHY_PATH, 'w') as f:
    json.dump(data, f, indent=4)